## Import packages

In [1]:
import numpy as np
import pandas as pd
import gc, random, math, time

In [2]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Dropout, Reshape, Flatten, Lambda, Permute
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate, Concatenate, multiply, Dot, dot
from keras.layers import  GlobalMaxPooling1D, GlobalAveragePooling1D, Input, SpatialDropout1D, Bidirectional
from keras.layers import CuDNNLSTM, CuDNNGRU, LSTM, GRU, Softmax
from keras import backend as K
from keras.engine.topology import Layer
from keras.preprocessing import sequence, text
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import optimizers

Using TensorFlow backend.


In [3]:
from sklearn.model_selection import KFold

## Hyper parameter setting

In [4]:
embed_size = 300 # how big is each word vector
max_features = 160000 # how many unique words to use (i.e num rows in embedding vector)
maxlen_p = 150 # max number of words in a context to use
maxlen_q = 15 # max number of words in a question to use
batch_size = 256
num_rnn_units = 64
num_hidden_units = 300
drop_prob = 0.5
max_norm = 5.0
features = 2

## File path

In [5]:
train_path = './data/train.tsv' # train set
valid_path = './data/valid.tsv' # validation set
test_path = './data/test.tsv' # test set
#embed_file = './sgns.target.word-ngram.1-2.dynwin5.thr10.neg5.dim300.iter5' # 预训练词向量
fasttext_file = './cc.zh.300.vec' # 预训练词向量
train_feature_p_path = './data/train_fea_p.npy' # train passage word feature
valid_feature_p_path = './data/valid_fea_p.npy' # validation passage word feature
test_feature_p_path = './data/test_fea_p.npy' # test passage word feature
train_feature_q_path = './data/train_fea_q.npy' # train passage word feature
valid_feature_q_path = './data/valid_fea_q.npy' # validation passage word feature
test_feature_q_path = './data/test_fea_q.npy' # test passage word feature

##  Read file

In [6]:
train = pd.read_csv(train_path, sep='\t', header=0)
valid = pd.read_csv(valid_path, sep='\t', header=0)
test = pd.read_csv(test_path, sep='\t', header=0)
print (train.shape, valid.shape, test.shape)
print (train.head())

(750000, 5) (90000, 5) (30000, 4)
   id                                            passage  \
0   1  孩子 是 父母 的 一面镜子   由于 儿童 的 世界观 尚未 形成   他们 的 模仿 带...   
1   1  孩子 是 父母 的 一面镜子   由于 儿童 的 世界观 尚未 形成   他们 的 模仿 带...   
2   1  孩子 是 父母 的 一面镜子   由于 儿童 的 世界观 尚未 形成   他们 的 模仿 带...   
3   2  目前   中国 很多 地方   学生 火车票 磁条 都 已经 升级 了   在 磁条 里 已...   
4   2  目前   中国 很多 地方   学生 火车票 磁条 都 已经 升级 了   在 磁条 里 已...   

                   query option  label  
0   你 的 孩子 无法确定 保姆 带 大 的   无法确定      1  
1      你 的 孩子 是 保姆 带 大 的      是      0  
2     你 的 孩子 不是 保姆 带 大 的     不是      0  
3  不是 一个 区间 刷 学生证 不能 有 票     不能      1  
4   不是 一个 区间 刷 学生证 能 有 票      能      0  


In [7]:
train_feature_p = np.load(train_feature_p_path)
valid_feature_p = np.load(valid_feature_p_path)
test_feature_p = np.load(test_feature_p_path)
print (train_feature_p.shape, valid_feature_p.shape, test_feature_p.shape)
train_feature_q = np.load(train_feature_q_path)
valid_feature_q = np.load(valid_feature_q_path)
test_feature_q = np.load(test_feature_q_path)
print (train_feature_q.shape, valid_feature_q.shape, test_feature_q.shape)

(750000, 150, 2) (90000, 150, 2) (30000, 150, 2)
(750000, 15, 2) (90000, 15, 2) (30000, 15, 2)


## Buld up the text input pipeline

####  Fit the tokenizer on train, valid and test set

In [8]:
tokenizer = Tokenizer(num_words=max_features, lower=True) 

tokenizer.fit_on_texts(pd.concat([train['passage'], train['query'], valid['passage'], valid['query'], test['passage'], test['query']], ignore_index=True))

In [9]:
print (tokenizer.document_count, len(tokenizer.word_counts))

1740000 167672


#### Data generator

In [ ]:
def train_gen(batch_size=batch_size):
    current = 0 # pointer
    p = train['passage']
    q = train['query']
    label = train['label']
    ind = random.shuffle(range(train.shape[0])) # shuffled index list
    
    while 1:
        if current + batch_size >= train.shape[0]:
            inds = ind[current:]
            current = 0
            # p, q
            p_batched = p.iloc[inds]
            q_batched = q.iloc[inds]
            seq_p = tokenizer.texts_to_sequences(p_batched)
            seq_q = tokenizer.texts_to_sequences(q_batched)
            pad_p = pad_sequences(seq_p, maxlen=maxlen_p)
            pad_q = pad_sequences(seq_q, maxlen=maxlen_q, padding='post', truncating='post')
            # label
            y = label.iloc[inds]
            # p, q features
            p_fea = train_feature_p[np.array(inds), :, :]
        else:
            batched = text.iloc[current : current + batch_size]
            batched_aug = text_aug(batched, label.iloc[current : current + batch_size])
            y = to_categorical(label.iloc[current : current + batch_size]-1, num_classes=19)
            current += batch_size
            
        tokenized = tokenizer.texts_to_sequences(batched_aug)
        padded = pad_sequences(tokenized, maxlen=maxlen)

        yield (padded, y)

## Prepare the pretrained word embedding

In [15]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(embed_file, encoding='utf-8'))

In [16]:
all_embs = np.hstack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.014820942, 0.26983637)

In [17]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words+1, embed_size))
for word, i in word_index.items():
    if i > max_features: break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [18]:
embedding_matrix = np.asarray(embedding_matrix, dtype='float32')

In [19]:
fasttext_index = dict(get_coefs(*o.strip().split()) for o in open(fasttext_file, encoding='utf-8'))
all_ft = np.hstack(fasttext_index.values())
ft_mean,ft_std = all_ft.mean(), all_ft.std()
fasttext_matrix = np.random.normal(ft_mean, ft_std, (nb_words+1, embed_size))
for word, i in word_index.items():
    if i > max_features: break
    fasttext_vector = fasttext_index.get(word)
    if fasttext_vector is not None: fasttext_matrix[i] = fasttext_vector
fasttext_matrix = np.asarray(fasttext_matrix, dtype='float32')

## Build the model

In [20]:
K.clear_session()

In [21]:
def attention_flow (x):
    h = x[0]
    u = x[1]
    #d = K.int_shape(h)[-1]
    #arr = K.variable(d, dtype='float32')
    #print (arr)
    s = K.batch_dot(h, K.permute_dimensions(u, (0,2,1)), axes=[2,1])  # [t, j]
    #print (s)
    p2q = K.batch_dot(K.softmax(s, axis=-1), u, axes=[2,1]) # [t, 2d]
    b = K.softmax(K.max(s, axis=-1, keepdims=True), -2) # [t, 1]
    q2p = K.tile(K.batch_dot(K.permute_dimensions(h, (0,2,1)), b, axes=[2,1]), [1, 1, K.int_shape(h)[1]]) # [2d, t]
    h_p2q = multiply([h, p2q]) # [t, 2d]
    h_q2p = multiply([h, K.permute_dimensions(q2p, (0,2,1))]) # [t, 2d]
    g = concatenate([h, p2q, h_p2q, h_q2p]) # [t, 8d]
    '''
    # self-attention
    sg = K.batch_dot(g, K.permute_dimensions(g, (0,2,1)), axes=[2,1]) # [t, t]
    g = K.batch_dot(K.softmax(sg, axis=-1), g, axes=[2,1]) # [t, 8d]
    '''
    return g

In [22]:
def cos_sim (x):
    p = x[0] # [t, 2d]
    q = x[1] # [j, 2d]
    s = dot([p, K.permute_dimensions(q, (0,2,1))], axes=(2,1), normalize=True) # [t, j] cosine simlilarity
    max_sim = K.max(s, axis=-1, keepdims=True) # [t, 1]
    return max_sim

In [29]:
def single_model():
    p = Input(shape=(maxlen_p,))
    q = Input(shape=(maxlen_q,))
    p_fea = Input(shape=(maxlen_p, features)) # passage word feature 
    q_fea = Input(shape=(maxlen_q, features)) # query word feature
    
    # Embedding layer
    embed = Embedding(nb_words+1, embed_size, weights=[embedding_matrix], trainable=True)
    ft = Embedding(nb_words+1, embed_size, weights=[fasttext_matrix], trainable=True)
    pem = embed(p) # word embedding
    pft = ft(p)
    pe = Concatenate()([pem, pft])
    qem = embed(q)
    qft = ft(q)
    qe = Concatenate()([qem, qft])
    
    p_cos_e = Lambda(cos_sim)([pem, qem])
    p_cos_f = Lambda(cos_sim)([pft, qft])
    q_cos_e = Lambda(cos_sim)([qem, pem])
    q_cos_f = Lambda(cos_sim)([qft, pft])
    pe = SpatialDropout1D(0.2)(pe)
    qe = SpatialDropout1D(0.2)(qe)
    pf = Concatenate()([pe, p_fea, p_cos_e, p_cos_f]) # passage feature vec = word embedding + (exact match + option match + cos sim)
    qe = Concatenate()([qe, q_fea, q_cos_e, q_cos_f]) # query feature vec = word embedding + (exact match + option match + cos sim)
    
    # Contextual embedding layer
    h = Bidirectional(CuDNNLSTM(num_rnn_units, return_sequences=True))(pf) # [t, 2d]
    u = Bidirectional(CuDNNLSTM(num_rnn_units, return_sequences=True))(qe) # [j,2d]
    
    # Attention flow layer
    g = Lambda(attention_flow)([h, u]) # [t, 8d]

    # Modelling layer
    m, hf, cf, hb, cb = Bidirectional(CuDNNLSTM(num_rnn_units, return_sequences=True, return_state=True))(g) # [t, 2d], d, d, d, d
    #m = Bidirectional(CuDNNLSTM(num_rnn_units, return_sequences=True))(m) # [t, 2d]
    
    um, uhf, ucf, uhb, ucb = Bidirectional(CuDNNLSTM(num_rnn_units, return_sequences=True, return_state=True))(u) # [j,2d], d, d, d, d

    # Output layer
    conc = Concatenate()([g, m]) # [t, 10d]
    gmp = GlobalMaxPooling1D()(conc) # [10d]
    gap = GlobalAveragePooling1D()(conc) # [10d]
    z1 = Concatenate()([gmp, gap, hf, hb]) # [22d]
    
    ugmp = GlobalMaxPooling1D()(um) # [4d]
    ugap = GlobalAveragePooling1D()(um) # [4d]
    z2 = Concatenate()([ugmp, ugap, uhf, uhb]) # [10d]

    y = Concatenate()([z1, z2])
    x = BatchNormalization()(y)
    x = Dense(num_hidden_units, activation='relu')(x)
    x = Dropout(drop_prob)(x)
    x = BatchNormalization()(x)
    #x = Dense(num_hidden_units, activation='relu')(x)
    #x = Dropout(drop_prob)(x)
    #x = BatchNormalization()(x)

    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[p, q, p_fea, q_fea], outputs=x)
    #print (model.summary())
    return model

## Train the model

In [30]:
model = single_model()

In [31]:
model.load_weights('./model/my6.h5')

In [32]:
adam = optimizers.Adam(lr=0.0002, clipnorm=max_norm)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['binary_accuracy'])
    
# train the model
cp = ModelCheckpoint(filepath='./model/my6.h5', monitor='val_binary_accuracy', save_best_only=True, save_weights_only=True)
es = EarlyStopping(patience=0,  monitor='val_binary_accuracy')
rp = ReduceLROnPlateau(patience = 1,  monitor='val_loss')
hist = model.fit(
    [train_p, train_q, train_feature_p, train_feature_q], 
    train_l,
    batch_size = batch_size,
    epochs = 4,
    shuffle = False,
    validation_data = ([valid_p, valid_q, valid_feature_p, valid_feature_q], valid_l), 
    callbacks=[rp, cp, es])

Train on 750000 samples, validate on 90000 samples
Epoch 1/4
750000/750000 [==============================] - 699s 932us/step - loss: 0.3451 - binary_accuracy: 0.8462 - val_loss: 0.4012 - val_binary_accuracy: 0.8218
Epoch 2/4
750000/750000 [==============================] - 696s 928us/step - loss: 0.3053 - binary_accuracy: 0.8667 - val_loss: 0.4219 - val_binary_accuracy: 0.8215


In [ ]:
print (hist.history)
from matplotlib import pyplot as plt
%matplotlib inline
plt.figure(1)
plt.plot (hist.history['binary_accuracy'])
plt.plot (hist.history['val_binary_accuracy'])

## load the best weight

In [ ]:
model.load_weights('./model/my6.h5')

## predict the test data

In [ ]:
test_pred = model.predict([test_p, test_q, test_feature_p, test_feature_q], batch_size=batch_size*4)

In [ ]:
test_pred = np.squeeze(test_pred)
print(test_pred.shape)

## Write the array into csv file

In [ ]:
res = pd.DataFrame({'id':test['id'], 'passage':test['passage'], 'query':test['query'], 'option':test['option'], 'label':test_pred})
res.to_csv('./result/test6_long.csv', index=False, encoding='utf-8_sig')

## check validation prediction result

val_pred = model.predict([valid_p, valid_q, valid_feature], batch_size=batch_size*4)
val_pred = np.squeeze(val_pred)
res = pd.DataFrame({'id':valid['id'], 'passage':valid['passage'], 'query':valid['query'], 'option':valid['option'], 'label':val_pred})
res.to_csv('./result/valid17_long.csv', index=False, encoding='utf-8_sig')